# Searching for best hyperparameters set

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.preprocessing import StandardScaler
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice
import plotly.express as px
import pickle
import os

In [ ]:
train_df = pd.read_feather('./data/train_processed.ftr')
val_df = pd.read_feather('./data/val_processed.ftr')
cols = list(train_df.columns)

In [ ]:
cols

## Prepare LGB Model

In [ ]:
train_x = train_df[cols[:-1]].to_numpy()
train_y = train_df[cols[-1]].to_numpy()

In [ ]:
val_x = val_df[cols[:-1]].to_numpy()
val_y = val_df[cols[-1]].to_numpy()

In [ ]:
class ModelTester:
    
    def __init__(self, params, train_x, train_y, val_x, val_y):
        self.params = params
        self.train_x = train_x
        self.train_y = train_y
        self.val_x = val_x
        self.val_y = val_y
        
        self.model = self.train_model()
        self.y_pred_bin = self.get_bin_predictions()
            
    def train_model(self):
        # 1 - Run model
        model = lgb.LGBMClassifier(
            boosting=self.params['boosting'],
            n_estimators=self.params['num_rounds'],
            max_depth=self.params['max_depth'],
            learnig_rate=self.params['learning_rate'],
            num_leaves = self.params['num_leaves'],
            min_child_samples = self.params['min_data_in_leaf'],
            subsample=self.params['bagging_fraction'],
            reg_alpha=self.params['lambda_l1'],
            reg_lambda=self.params['lambda_l2']
        )
        model.fit(train_x, train_y)
        return model
    
    def get_bin_predictions(self):
        # 2 - Get predictions
        y_pred = self.model.predict(val_x)
        y_pred_bin = [1 if y >= 0.5 else 0 for y in y_pred]
        
        return y_pred_bin
    
    def get_accuracy(self):
        return round(accuracy_score(self.val_y, self.y_pred_bin), 3)
    
    def get_class_report(self):
        return classification_report(self.val_y, self.y_pred_bin)    

## Optuna research

In [ ]:
def objective(trial):
    # 0 - Prepare params
    num_rounds = trial.suggest_int("num_rounds", 10, 100)
    boosting = trial.suggest_categorical('boosting', ['gbdt', 'dart'])
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.5)
    max_depth = trial.suggest_int('max_depth', 5, 100)
    num_leaves = trial.suggest_int("num_leaves", 2, 100)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 10, 500)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.9, 1.)
    extra_trees = trial.suggest_categorical('extra_trees', [True, False])
    lambda_l1 = trial.suggest_float('lambda_l1', 0, 0.1)
    lambda_l2 = trial.suggest_float('lambda_l2', 0, 0.1)
    
    params = {
        'num_rounds': num_rounds,
        'objectives':'binary',
        'verbosity':0,
        'boosting':boosting,
        'learning_rate':learning_rate,
        'max_depth':max_depth,
        'num_leaves':num_leaves,
        'min_data_in_leaf':min_data_in_leaf,
        'bagging_fraction':bagging_fraction,
        'extra_trees':extra_trees,
        'lambda_l1':lambda_l1,
        'lambda_l2':lambda_l2
    }
    
    # 1 - Model
    model_tester = ModelTester(params, train_x, train_y, val_x, val_y)
    acc = model_tester.get_accuracy()
    error = 1 - acc
    
    return error

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=1000)

### Show results

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
key_params = ['num_leaves', 'min_data_in_leaf', 'num_rounds', 'learning_rate', 'max_depth']
plot_slice(study, params = key_params).show()

In [ ]:
study.best_params

In [ ]:
best_params = {
'num_rounds': 89,
'boosting': 'gbdt',
'learning_rate': 0.19673057125087745,
'max_depth': 88,
'num_leaves': 8,
'min_data_in_leaf': 37,
'bagging_fraction': 0.9514661346499478,
'extra_trees': False,
'lambda_l1': 0.08212421594574758,
'lambda_l2': 0.0021191892176790073
}

## Create model with best params

In [ ]:
model = lgb.LGBMClassifier(
    boosting=best_params['boosting'],
    n_estimators=best_params['num_rounds'],
    max_depth=best_params['max_depth'],
    learnig_rate=best_params['learning_rate'],
    num_leaves=best_params['num_leaves'],
    min_child_samples=best_params['min_data_in_leaf'],
    subsample=best_params['bagging_fraction'],
    reg_alpha=best_params['lambda_l1'],
    reg_lambda=best_params['lambda_l2']
)

In [ ]:
X = np.concatenate((train_x, val_x))
y = np.concatenate((train_y, val_y))

In [ ]:
cv_scores = cross_val_score(model, X, y, cv=10)
print(f'Mean cv (k=10) accuracy score = {round(np.mean(cv_scores), 3)}')

## Feature selection

In [ ]:
is_cat_col = {
    'is_g734s': True,
    'CryoSleep': True,
    'VIP': True,
    'Europa': True,
    'Mars': True,
    'PSO J318.5-22': True,
    'TRAPPIST-1e': True,
    'Age': False,
    'RoomService': False,
    'FoodCourt': False,
    'ShoppingMall': False,
    'Spa': False,
    'VRDeck': False,
    'segment': True
 }

feature_mi = mutual_info_classif(X, y, discrete_features=list(is_cat_col.values()))
feature_f = f_classif(X, y)

feature_df = pd.DataFrame({
    'feature': cols[:-1],
    'mi': feature_mi,
    'f': feature_f[0]
})

scaler = StandardScaler()
feature_df[['mi_z', 'f_z']] = scaler.fit_transform(feature_df[['mi', 'f']])
feature_df['feature_quality'] = feature_df[['mi_z', 'f_z']].mean(axis=1)

feature_df.sort_values('feature_quality', ascending=False, inplace=True)
best_features = feature_df['feature'].to_list()
feature_df

In [ ]:
fig = px.bar(feature_df, x='feature_quality', y='feature')
fig.show()

In [ ]:
temp_cols = []
cv_list = []

model = lgb.LGBMClassifier(
    boosting=best_params['boosting'],
    n_estimators=best_params['num_rounds'],
    max_depth=best_params['max_depth'],
    learnig_rate=best_params['learning_rate'],
    num_leaves=best_params['num_leaves'],
    min_child_samples=best_params['min_data_in_leaf'],
    subsample=best_params['bagging_fraction'],
    reg_alpha=best_params['lambda_l1'],
    reg_lambda=best_params['lambda_l2']
)

for feature in best_features:
    temp_cols.append(feature)
    X = pd.concat([train_df[temp_cols], val_df[temp_cols]]).to_numpy()
    cv_score = cross_val_score(model, X, y, cv=10)
    cv_list.append(cv_score)

In [ ]:
means = []
for scores in cv_list:
    means.append(round(np.mean(scores), 3))

i_features = 0
for i in range(len(means)):
    if means[i] == max(means):
        i_features = i
        break

In [ ]:
best_features

# Make final model

In [ ]:
train_x = train_df[best_features[:i_features+1]].to_numpy()
val_x = val_df[best_features[:i_features+1]].to_numpy()

In [ ]:
model = lgb.LGBMClassifier(
    boosting=best_params['boosting'],
    n_estimators=best_params['num_rounds'],
    max_depth=best_params['max_depth'],
    learnig_rate=best_params['learning_rate'],
    num_leaves=best_params['num_leaves'],
    min_child_samples=best_params['min_data_in_leaf'],
    subsample=best_params['bagging_fraction'],
    reg_alpha=best_params['lambda_l1'],
    reg_lambda=best_params['lambda_l2']
)

model.fit(train_x, train_y)
pred_val = model.predict_proba(val_x)
pred_train = model.predict_proba(train_x)

In [ ]:
model_dict = {
    'model': model,
    'features': best_features
}

filepath = os.path.join('models', 'lgbm.pickle')
with open(filepath, 'wb') as file:
    pickle.dump(model_dict, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
train_proba_df = pd.DataFrame(pred_train)
train_proba_df.columns = ['0', 'lgbm']
train_proba_df['y'] = train_y.astype(int)
train_proba_df.drop(columns='0', inplace=True)
train_proba_df

In [ ]:
val_proba_df = pd.DataFrame(pred_val)
val_proba_df.columns = ['0', 'lgbm']
val_proba_df['y'] = val_y.astype(int)
val_proba_df.drop(columns='0', inplace=True)
val_proba_df

In [ ]:
train_proba_final = pd.read_csv('./data/ensemble_train_df.csv')
train_proba_final['lgbm'] = train_proba_df['lgbm']
val_proba_final = pd.read_csv('./data/ensemble_val_df.csv')
val_proba_final['lgbm'] = val_proba_df['lgbm']

In [ ]:
train_proba_final = train_proba_final[['ada_boost', 'svc', 'lgbm', 'y']]
train_proba_final.to_feather('./data/ensemble_train_df.ftr')
val_proba_final = val_proba_final[['ada_boost', 'svc', 'lgbm', 'y']]
val_proba_final.to_feather('./data/ensemble_val_df.ftr')

In [ ]:
filepath = os.path.join('models', 'lgbm.pickle')
pickle.dump(model, open(filepath, 'wb'))